# 기상 데이터 전처리 

## 기상 데이터 전처리 (최고기온)
- 목표 : 제주도 지역별 최고기온 데이터를 가져온 후, 조건에 해당하는 값을 예측값으로 지정, 최고기온 및 습도, 강수 확률, 강수 형태 데이터를 합쳐서 하나의 데이터로 가공

[데이터 원본 사이트]
https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

[변수 설명]
- ara : 아라동(제주시) 최고기온 데이터
- cheonji : 천지동(서귀포시) 최고기온 데이터


[칼럼 설명]
- format: day : 일자 (몇일인지 ex. 1,2,3,4...31)	
- hour	: 예측한 시간 (영국 그리니치 천문대(UTC) 기준 -> 실제 한국 시간에 반영하기 위해서는 시차인 9시간을 반영해야 함.) 
- forecast : hour 칼럼 기준 몇 시간 후의 온도를 예측할 것인지 지정. (ex. forecast의 값이 4이고, hour이 200이라면, UTC 오전 2시 기준, 즉 오전 11시 기준으로 4시간 후의 값을 예측)
- value location:53_37 Start:20210101 : 최고기온 칼럼


[습도, 강수 확률, 강수 형태 데이터 칼럼 설명]
- humidity(%) : 습도
- 강수 확률 : 강수 확률 (단위 : %)
- 강수 형태 : 강수 형태 (0 -> 강수 없음, 1 -> 비, 2 -> 눈, 3 -> 눈비)



[지정할 조건]
- Hour 칼럼이 200이고 forecast가 7인 조건을 모든 데이터에 공통으로 적용
    - UTC  시간 기준 2:00이고, 7시간 뒤의 예측값 지정


[유의사항]
- 2021년 전반기(1~6월)의 경우에는 검색조건의 구분에서 단기예보를 (구)단기예보로 전환해줘야 확인 가능.
- 제주시의 경우 아라동, 서귀포시의 경우는 천지동을 대표 데이터로 설정하여 전처리 과정 진행
- 단, 강수 형태는 서귀포시만 데이터를 받았음, 서귀포시 데이터만 합침

[과정]
- 데이터 원본 사이트 (기상자료개방포털-단기예보)에서 아라동, 천지동의 2021년의 최고기온 추출
- 조건에 해당하는 칼럼만 추출한 후,데이터프레임으로 저장
- 습도, 강수 확률, 강수 형태 데이터도 불러온 후 새로운 데이터프레임으로 만들어 저장 (제주시는 강수 형태 제외)


In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

In [2]:
# 날씨 데이터 가져오기
df1 = pd.read_csv('../Dataset/raw_data/weather/아라동 전반기.csv') # 제주시 1~6월
df2 = pd.read_csv('../Dataset/raw_data/weather/아라동 후반기.csv') # 제주시 7~12월
df3 = pd.read_csv('../Dataset/raw_data/weather/천지동 전반기.csv') # 서귀포시 1~6월
df4 = pd.read_csv('../Dataset/raw_data/weather/천지동 후반기.csv') # 서귀포시 7~12월

In [3]:
# 대표로 아라동 전반기의 데이터 확인
df1

,format: day,hour,forecast,value location:53_37 Start : 20210101
0,1,200.0,4.0,4.0
1,1,200.0,7.0,5.0
2,1,200.0,10.0,3.0
3,1,500.0,4.0,5.0
4,1,500.0,7.0,3.0
...,...,...,...,...
3440,30,2000.0,10.0,26.0
3441,30,2300.0,4.0,28.0
3442,30,2300.0,7.0,27.0
3443,30,2300.0,10.0,26.0


In [4]:
# 칼럼명 재지정(제주시, 서귀포시 모든 데이터)
df1.columns = ['일','UTC','예측+시간','최고기온']
df2.columns = ['일','UTC','예측+시간','최고기온']
df3.columns = ['일','UTC','예측+시간','최고기온']
df4.columns = ['일','UTC','예측+시간','최고기온']

In [5]:
# 조건에 맞는 칼럼만 가져오기
# 조건 : UTC->200, 예측+시간->7.0 (그리니치 천문대 기준 오전 2시, 즉 한국 기준 11시에서 예측한 7시간 뒤의 기온 -> 오후 6시 기온)
fc1 = df1[(df1['UTC']==200.0) & (df1['예측+시간']==7.0)] # 제주시 전반기
fc2 = df2[(df2['UTC']==200.0) & (df2['예측+시간']==7.0)] # 제주시 후반기
fc3 = df3[(df3['UTC']==200.0) & (df3['예측+시간']==7.0)] # 서귀포시 전반기
fc4 = df4[(df4['UTC']==200.0) & (df4['예측+시간']==7.0)] # 서귀포시 후반기

In [6]:
# 길이 확인 (합쳐서 365개가 맞아야 함)
len(fc1), len(fc2)

(181, 184)

In [7]:
# 길이 확인 (합쳐서 365개가 맞아야 함)
len(fc3), len(fc4)

(181, 184)

In [8]:
# 함수를 이용해서 2021-01-01부터 2021-12-31까지 모든 데이터 불러옴
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2021-01-01", "2021-12-31")
# 대표적으로 앞의 5개만 추출
date_list[:5]

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05']

In [9]:
# 아라동(제주시) 데이터 합치고 날짜 칼럼에 DATE_LIST 저장
ara = pd.concat([fc1,fc2])
# date_list를 ara 데이터의 날짜 칼럼에 저장
ara['날짜'] = date_list 
# 대표적으로 앞의 5개만 확인
ara.head()

,일,UTC,예측+시간,최고기온,날짜
1,1,200.0,7.0,5.0,2021-01-01
20,2,200.0,7.0,4.0,2021-01-02
39,3,200.0,7.0,7.0,2021-01-03
58,4,200.0,7.0,5.0,2021-01-04
77,5,200.0,7.0,2.0,2021-01-05


In [10]:
# 아라동(제주시) 최고기온 결과물 저장
ara.to_csv('../dataset/raw_data/weather/아라동일별기온.csv',encoding='utf-8-sig',index=False)

In [11]:
# 천지동(서귀포시) 데이터 합치고 날짜 칼럼에 DATE_LIST 저장
cheonji = pd.concat([fc3,fc4])
# date_list를 choenji 데이터의 날짜 칼럼에 저장
cheonji['날짜'] = date_list
# 대표적으로 앞의 5개만 확인
cheonji.head()

,일,UTC,예측+시간,최고기온,날짜
1,1,200.0,7.0,11.0,2021-01-01
20,2,200.0,7.0,10.0,2021-01-02
39,3,200.0,7.0,12.0,2021-01-03
58,4,200.0,7.0,12.0,2021-01-04
77,5,200.0,7.0,8.0,2021-01-05


In [12]:
# shape 확인
cheonji.shape

(365, 5)

In [13]:
# 천지동(서귀포시) 최고기온 데이터 결과물 저장
cheonji.to_csv('../Dataset/raw_data/weather/천지동일별기온.csv',encoding='utf-8-sig',index=False)

In [14]:
# 중복되는 날짜가 있는지 nunique()를 통해 확인
ara.nunique()

일         31
UTC        1
예측+시간      1
최고기온      35
날짜       365
dtype: int64

In [15]:
# 중복되는 날짜가 있는지 nunique()를 통해 확인
cheonji.nunique()

일         31
UTC        1
예측+시간      1
최고기온      30
날짜       365
dtype: int64

## 다른 데이터 결합

In [16]:
# 습도 데이터 불러오기
df5 = pd.read_csv('../Dataset/raw_data/weather/jeju.csv') # 제주시 습도
df6 = pd.read_csv('../Dataset/raw_data/weather/seogwipo.csv') # 서귀포시 습도

In [17]:
# 제주시 습도 확인
df5

,날짜,공휴일,요일,humidity(%)
0,2021-01-01,1,금,湲?75
1,2021-01-02,0,토,??75
2,2021-01-03,0,일,??75
3,2021-01-04,0,월,??75
4,2021-01-05,0,화,??75
...,...,...,...,...
360,2021-12-27,0,월,??80
361,2021-12-28,0,화,??85
362,2021-12-29,0,수,??75
363,2021-12-30,0,목,紐?65


In [18]:
# humidity(%) 칼럼이 깨져서 나왔기 때문에, 숫자만 남기는 정규표현식으로 숫자만 남기기
import re
only_num = re.compile('[^0-9]') # 숫자를 제외하고 나머지는 제거하는 정규표현식
for i in range(len(df5)):
    df5['humidity(%)'][i] = only_num.sub(' ',df5['humidity(%)'][i]) # 제주시 습도 데이터를 변환
    df6['humidity(%)'][i] = only_num.sub(' ',df6['humidity(%)'][i]) # 서귀포시 습도 데이터를 변환

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_13848\343642116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['humidity(%)'][i] = only_num.sub(' ',df5['humidity(%)'][i]) # 제주시 습도 데이터를 변환
C:\Users\LeeSH\AppData\Local\Temp\ipykernel_13848\343642116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['humidity(%)'][i] = only_num.sub(' ',df6['humidity(%)'][i]) # 서귀포시 습도 데이터를 변환


In [19]:
# 제주시 - 아라동, 서귀포시 - 천지동의 최고기온 저장
df5['최고기온'] = list(ara['최고기온']) # 아라동의 최고기온을 df5 (제주시 습도 데이터)에 합침
df6['최고기온'] = list(cheonji['최고기온']) # 천지동의 최고기온을 df5 (서귀포시 습도 데이터)에 합침

In [20]:
# csv 형태로 저장
df5.to_csv('../Dataset/raw_data/weather/제주시날씨.csv',encoding='utf-8-sig',index=False) # 제주시 날씨 (습도 + 최고기온)
df6.to_csv('../Dataset/raw_data/weather/서귀포시날씨.csv',encoding='utf-8-sig',index=False) # 서귀포시 날씨 (습도 + 최고기온)

In [21]:
# 강수확률 데이터 가져오기
df7 = pd.read_csv('../Dataset/raw_data/weather/아라동강수확률.csv') # 제주시(아라동) 강수확률
df8 = pd.read_csv('../Dataset/raw_data/weather/천지동강수확률.csv') # 서귀포시(천지동) 강수확률

In [22]:
# 제주시 강수확률 데이터 확인
df7

,날짜,일,강수확률
0,2021-01-01,1,30
1,2021-01-02,2,30
2,2021-01-03,3,30
3,2021-01-04,4,0
4,2021-01-05,5,30
...,...,...,...
360,2021-12-27,27,30
361,2021-12-28,28,30
362,2021-12-29,29,30
363,2021-12-30,30,60


In [23]:
# 기존의 데이터(df5, df6)에 강수확률 칼럼 저장
df5['강수확률']=list(df7['강수확률'])
df6['강수확률']=list(df8['강수확률'])

In [24]:
# 칼럼 순서 지정 및 특정 칼럼만 가져오기
df5 = df5[['날짜','공휴일','요일','humidity(%)','강수확률','최고기온']]
df6 = df6[['날짜','공휴일','요일','humidity(%)','강수확률','최고기온']]

In [25]:
# 결과파일 csv 형태로 저장
df5.to_csv('../Dataset/raw_data/weather/jeju_weather.csv',encoding='utf-8-sig',index=False) # 제주시 데이터 (습도, 강수확률, 최고기온)
df6.to_csv('../Dataset/raw_data/weather/seogwipo_weather.csv',encoding='utf-8-sig',index=False) # 서귀포시 데이터 (습도, 강수확률, 최고기온)

In [26]:
# 서귀포시 강수형태 데이터 가져오기
df9 = pd.read_csv('../Dataset/raw_data/weather/천지동강수형태.csv')
# 데이터 확인
df9

,날짜,일,강수형태
0,2021-01-01,1,0
1,2021-01-02,2,0
2,2021-01-03,3,0
3,2021-01-04,4,0
4,2021-01-05,5,0
...,...,...,...
360,2021-12-27,28,0
361,2021-12-28,29,0
362,2021-12-29,30,0
363,2021-12-30,31,0


In [27]:
# 기존에 합쳐놓았던 데이터(df6에 강수형태 칼럼 저장
df6['강수형태'] = list(df9['강수형태'])

In [28]:
# 결과 확인
df6

,날짜,공휴일,요일,humidity(%),강수확률,최고기온,강수형태
0,2021-01-01,1,금,60,30,11.0,0
1,2021-01-02,0,토,70,20,10.0,0
2,2021-01-03,0,일,70,20,12.0,0
3,2021-01-04,0,월,60,0,12.0,0
4,2021-01-05,0,화,65,20,8.0,0
...,...,...,...,...,...,...,...
360,2021-12-27,0,월,65,20,11.0,0
361,2021-12-28,0,화,70,30,13.0,0
362,2021-12-29,0,수,60,30,9.0,0
363,2021-12-30,0,목,50,20,8.0,0


In [29]:
# csv 형태로 저장
df6.to_csv('../Dataset/raw_data/weather/seogwipo_weather.csv',encoding='utf-8-sig',index=False) # 서귀포시 최종 날씨 데이터

In [30]:
# 아라동(제주시) 데이터에서 일별로 개수가 다른지, 문제는 없는지 파악
ara['일'].value_counts()

1     12
2     12
28    12
27    12
26    12
25    12
24    12
23    12
22    12
21    12
20    12
19    12
18    12
17    12
16    12
15    12
14    12
13    12
12    12
11    12
10    12
9     12
8     12
7     12
6     12
5     12
4     12
3     12
29    11
30    11
31     7
Name: 일, dtype: int64

In [31]:
# 천지동(서귀포시) 데이터에서 일별로 개수가 다른지, 문제는 없는지 파악
cheonji['일'].value_counts()

1     12
2     12
28    12
27    12
26    12
25    12
24    12
23    12
22    12
21    12
20    12
19    12
18    12
17    12
16    12
15    12
14    12
13    12
12    12
11    12
10    12
9     12
8     12
7     12
6     12
5     12
4     12
3     12
29    11
30    11
31     7
Name: 일, dtype: int64